# 크롤링 자동화

In [33]:
import pandas as pd  
import numpy as np   

import requests
from bs4 import BeautifulSoup    
from selenium import webdriver   
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller 

import time    
import re
import math
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

In [3]:
pwd

'c:\\Users\\Jay\\Desktop\\python_ML'

In [39]:
df = pd.read_excel('gyobo_best_all.xlsx') 
df = df[df['담당자']==1] # 담당자 변경
df.reset_index(drop=True, inplace= True)
df

,순위,바코드,도서명,저자명,출판사명,출간일,정가,카테고리,담당자
0,101,9791190641159,첫 번째 디자인 케이크,지은혜(아이라이크케이크),책밥,20200820,16000,요리,1
1,79,9788982814471,연금술사,파울로 코엘료,문학동네,20181205,12000,소설,1
2,138,9791189727451,기계설비 표준품셈(2022)(CD1장포함)(양장본 HardCover),건설연구원,건설연구원,20220114,34000,기술/공학,1
3,115,9791167900791,그때 그 마음(2022 제67회 현대문학상 수상소설집),정소현 외,현대문학,20211210,15000,소설,1
4,89,9791165343842,대한민국 금기 깨기,김동연,쌤앤파커스,20210728,17000,정치사회,1
...,...,...,...,...,...,...,...,...,...
915,52,3904000300085,뉴필로소퍼 (2022년 Vol. 17),뉴필로소퍼 편집부,바다출판사,20220106,15000,잡지,1
916,43,7051769003744,요시타케 신스케 작가 시리즈 전8권 세트(창작동화2권 증정) : 신간 심심해 심심해 포함,요시타케 신스케,주니어김영사/스콜라/봄나무,20170830,91000,어린이전집,1
917,326,9780201825930,Exploring English 1.(Work Book),"Harris, Tim",Prentice-Hall,19950901,9500,어린이이엘티,1
918,65,9788998441074,나는 소망한다 내게 금지된 것을,양귀자,쓰다,20190420,15000,소설,1


In [35]:
df_keyword = pd.DataFrame( columns=['id', 'klover','comment','keyword','score','total','title'])
df_keyword

,id,klover,comment,keyword,score,total,title


In [36]:
## 전처리 함수 완성 ## 
def preprocessing(total):
    k_total = int(re.sub('\D','',total))
    return k_total

def keyword_pre(keywords):
    for keyword in keywords:
        keyword_list.append(keyword.text)

def id_pre(ids):
    for id in ids:
        id_list.append(id.text)
        
def klovers_pre(klovers):
    for klover in klovers:
        klover_list.append(klover.text)
        
def comments_pre(comments):
    for comment in comments:
        comment_list.append(comment.text)

In [40]:

for j in tqdm_notebook(range(156, 157)):
    url = df.iloc[j][1]
    title = df['도서명'][j]

# 크롬 웹 브라우저 실행 (keyword 검색결과)

    chrome_path = chromedriver_autoinstaller.install()
    driver = webdriver.Chrome(chrome_path)
    driver.get('http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&linkClass=01&barcode={}'.format(url)) # query : 바코드 숫자
    time.sleep(1)
    
    # 회원리뷰 클릭 : 그래야 clova 회원리뷰 목록이 나옴
    try:
        driver.find_element_by_partial_link_text('회원리뷰').click() 
        time.sleep(3)
    
    ## 전처리 함수 완성 ## 

        keyword_list = []
        id_list = []
        klover_list = []
        comment_list = []
    
        total = driver.find_element_by_id('kloverTotal').text
        k_total = preprocessing(total)
        page_n = math.ceil(k_total / 5)
        n =page_n -page_n % 10
        m= page_n %10+3
    
        # 키워드 수집
        keywords = driver.find_elements_by_class_name('book_keyword > a')
        keyword_pre(keywords)

        # 스코어 수집 
        score = driver.find_element_by_class_name('score').text


        i = 0
    

        while i < page_n :
            
    
            ids = driver.find_elements_by_css_selector('.box_detail_review .board_list .comment_wrap .id')
            id_pre(ids)
   
            klovers = driver.find_elements_by_css_selector('.comment_wrap .kloverRating > span')
            klovers_pre(klovers)

            comments = driver.find_elements_by_css_selector('.box_detail_review .board_list .comment_wrap .comment')
            comments_pre(comments)
    
    
            i +=1
            time.sleep(1) 
    
            if i> n or k_total <= 50:
                index = m
            else:
                index = 13 
    
            if i < page_n:
        
                css_selector= '.box_detail_review > div.list_paging.align_center > div > a:nth-child({})'.format(index)
                driver.find_element_by_css_selector(css_selector).click()
            else:
                print(j,'th done:',title)
                driver.close()

            time.sleep(1)
            
            data = [(id_list,klover_list,comment_list)]
            lable = ['id','klover','comment']
            df_temp= pd.DataFrame(zip(id_list,klover_list,comment_list),columns=lable)
           
            # 키워드, 스코어, 리뷰 수 컬럼 추가
            keyword_col = ' '.join(keyword_list)
            df_temp['keyword'] = keyword_col
            df_temp['score']= score
            df_temp['total'] = k_total
            df_temp['title'] = title
            
            # 도서명 그대로 저장
            
            title.replace('.','')
            title.replace('/','')
            df_temp.to_excel('{}.xlsx'.format(title))
            
    except:
            driver.close()
            print(j,'th error:',title)
    
    df_keyword = df_keyword.append([df_temp.iloc[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

156 th done: 된다! 하루 5분 노션 활용법: 포트폴리오, 개인 관리, 업무 협업


In [32]:
df_keyword


,id,klover,comment,keyword,score,total,title
0,eu**ye1103,4,"어떤것에 대해서 누가 발명했는지, 왜 발명했는지 생각해본 적 있으신가요? 발명과 과...",발명 글 상자 이모티콘 역사 플래시 인류,10.0,2,맛있고 재밌고 편리한 것들의 기원과 원리 100(어린이 미래 교양 시리즈 15)
0,jo**wea,4,MZ시대의 작가로 신경유전학의 어려운 부분을 일상속의 예를 연결하여 아주 읽기 편하...,뇌과학 유전학 신경 과학 교양과학 다윈 나폴레옹,10.0,2,아주 긴밀한 연결
0,bl**s70a,4,"왜 일본이 메이지유신을 통해 발전할 수 있었는지, 왜 조선 정복을 하려고 했는지 알...",조선 근대 일본 일본사 료마 일본근현대사 사카모토,9.8,27,메이지유신을 설계한 최후의 사무라이들(서가명강 14)
0,bl**s70a,4,"왜 일본이 메이지유신을 통해 발전할 수 있었는지, 왜 조선 정복을 하려고 했는지 알...",조선 근대 일본 일본사 료마 일본근현대사 사카모토,9.8,27,메이지유신을 설계한 최후의 사무라이들(서가명강 14)
0,tt**k1009,3,"중2여학생 독서코칭 도서인데, 재밌다고하네요~",과학 청소년과학 의학 실험 수용소 민낯,9.4,16,나쁜 과학자들
...,...,...,...,...,...,...,...
0,me**nation,4,초등학교 학생을 둔 학부모가 수포자가 되지 않게 하는 방어 책입니다.,질문 수능 성적 이유 불꽃 아이 공부,10.0,1,초등생을 위한 수학 공부몸 만들기(바른 교육 시리즈 20)
0,fl**ia61,4,아이들과 생각 너눔 이야기거리가 많아 졌어요.,,10.0,1,요시타케 신스케 작가 시리즈 전8권 세트(창작동화2권 증정) : 신간 심심해 심심해 포함
0,fl**ia61,4,아이들과 생각 너눔 이야기거리가 많아 졌어요.,,10.0,1,요시타케 신스케 작가 시리즈 전8권 세트(창작동화2권 증정) : 신간 심심해 심심해 포함
0,ma**9184,4,"삶에 대해 사유하고 싶은 모든 이, 특히 여성을 위한 책. 기발하고 짜릿하지만 동시...",페미니즘 한국현대소설 장편소설 여성 억압 장편 성불평등,9.8,150,나는 소망한다 내게 금지된 것을


In [23]:
title

'Tripful(트립풀) 런던(Tripful 시리즈 7)'

<bound method DataFrame.duplicated of            id klover                                            comment  \
0  eu**ye1103      4  어떤것에 대해서 누가 발명했는지, 왜 발명했는지 생각해본 적 있으신가요? 발명과 과...   
0     jo**wea      4  MZ시대의 작가로 신경유전학의 어려운 부분을 일상속의 예를 연결하여 아주 읽기 편하...   
0    bl**s70a      4  왜 일본이 메이지유신을 통해 발전할 수 있었는지, 왜 조선 정복을 하려고 했는지 알...   
0    bl**s70a      4  왜 일본이 메이지유신을 통해 발전할 수 있었는지, 왜 조선 정복을 하려고 했는지 알...   
0   tt**k1009      3                          중2여학생 독서코칭 도서인데, 재밌다고하네요~   
0  do**ydj103      4                                       상세하게 잘 나와있어요   
0  do**ydj103      4                                       상세하게 잘 나와있어요   
0     nk**118      4                                   법률 용어에 대해 적어둔 사전   
0    sa**aeul      4                                 좋은책이 나왔네요 아이가 좋아해요   
0  lu**lu2000      4  새로운 곡들이 많아진거같진 않지만 어쨌든 중단된줄 알았던 출판이 이어져서 다행이네요...   
0  lu**lu2000      4  새로운 곡들이 많아진거같진 않지만 어쨌든 중단된줄 알았던 출판이 이어져서 다행이네요...   
0  lu**lu2000      4  새로운 곡들이 많아진거같진 않지만 어쨌든 중단된줄 알았던 출판이 이어져서

FileNotFoundError: [Errno 2] No such file or directory: '사용자를 사로잡는 UX/UI 실전가이드.xlsx'